In [ ]:
import pandas as pd
import requests
from tqdm import tqdm
import time

## ETL search. 
### Se busca Ipone 11 Usados, Samsungs S20 Usados y Motorola Edge Usados

Se exporta como .csv con el nombre "Table_search.csv"


In [ ]:

URLs = [f'https://api.mercadolibre.com/sites/MLA/search?q=Iphone%2011&ITEM_CONDITION=2230581', #Iphone11 Usados
        'https://api.mercadolibre.com/sites/MLA/search?q=Samsung%20S20&ITEM_CONDITION=2230581', #Samsung S20 Usados
        'https://api.mercadolibre.com/sites/MLA/search?q=Motorola%20Edge&ITEM_CONDITION=2230581'] #Motorola Edge Usados

df_search_c = pd.DataFrame()

for x in tqdm(URLs):
    req_search = requests.get(x)
    search_items = req_search.json()
    df_search = pd.json_normalize(search_items, record_path =['results'], 
                                     meta=['query',['paging','total'],['paging','limit']])
    df_search_c = pd.concat([df_search_c, df_search], ignore_index=True);
    
df_search_c.to_csv('C:\Diego\MELI\Table_search.csv',encoding='utf-8-sig')




## ETL items. 
### A partir de la busqueda anterior se obtienen los diferentes Items de forma particular. 

Se exporta como .csv con el nombre "Table_items.csv"


In [ ]:
df_list_items = df_search_c['id'].tolist()
URL_item = r'https://api.mercadolibre.com/items/%s'

df_items_c = pd.DataFrame()

for n in tqdm(df_list_items):
    URL_PR = URL_item % n
    req_items = requests.get(URL_PR)
    files_items = req_items.json()
    df_items = pd.json_normalize(files_items)
    df_items_c = pd.concat([df_items_c, df_items], ignore_index=True)

df_items_c.to_csv('C:\Diego\MELI\Table_items.csv',encoding='utf-8-sig')
    
# URL_2 = f'https://api.mercadolibre.com/items/MLA916135036'

# r_2 = requests.get(URL_2)
# files_2 = r_2.json()
# df_2 = pd.DataFrame(files_2['sale_terms'])

#df_nested_list = pd.json_normalize(files_2)


## ETL items.
### A partir de la busqueda anterior se obtienen las preguntas realizadas.

Se exporta como .csv con el nombre "Table_questions.csv"

In [ ]:
df_list_items = df_search_c['id'].tolist()
URL_q = r'https://api.mercadolibre.com/questions/search?item=%s'

df_q_c = pd.DataFrame()

for n in tqdm(df_list_items):
    URL_PR_q = URL_q % n
    q_items = requests.get(URL_PR_q)
    files_q = q_items.json()
    s = 'status' in files_q
    while ((s == True) and (files_q['status'] == 429)):    ##Reintentos frente a errores de consultas
             q_items = requests.get(URL_PR_q)
             files_q = q_items.json()
             s = 'status' in files_q
    df_q = pd.json_normalize(files_q)
    df_q_c = pd.concat([df_q_c, df_q], ignore_index=True)


    
df_q_csv = df_q_c[['total','limit','filters.item']]

df_q_csv.to_csv('C:\Diego\MELI\Table_questions.csv',encoding='utf-8-sig',sep=',')


## ETL items.
### A partir de la busqueda anterior se obtienen los usuarios que realizaron las publicaciones (sellers).

Se exporta como .csv con el nombre "Table_users.csv"

In [ ]:
df_list_users = df_items_c['seller_id'].tolist()

URL_users = r'https://api.mercadolibre.com/users/%s'

df_users_c = pd.DataFrame()

for n in tqdm(df_list_users):
    URL_PR = URL_users % n
    req_users = requests.get(URL_PR)
    files_users = req_users.json()
    df_users = pd.json_normalize(files_users)
    df_users_c = pd.concat([df_users_c, df_users], ignore_index=True)

df_users_c.to_csv('C:\Diego\MELI\Table_users.csv',encoding='utf-8-sig')

In [ ]:
df_search_c.describe()